In [15]:
# Pfad zu Markdown File festlegen

#extract = "origin"
extract = "geschichtsbuch"

markdown_path_origin = "docs/extracted_content_dan_brown.md"
markdown_path_path_geschichtsbuch = "docs/extracted_content_geschichtsbuch.md" 

json_chunk_origin = "docs/chunks_dan_brown.json"
json_chunk_geschichtsbuch = "docs/chunks_geschichtsbuch.json"

if extract == "geschichtsbuch": 
    markdown_path = markdown_path_path_geschichtsbuch
    json_chunk_path = json_chunk_geschichtsbuch
else:
    markdown_path = markdown_path_origin
    json_chunk_path = json_chunk_origin

print(f"markdown_path is: {markdown_path}")
print(f"json_chunk_path is: {json_chunk_path}")


markdown_path is: docs/extracted_content_geschichtsbuch.md
json_chunk_path is: docs/chunks_geschichtsbuch.json


## Parameter für Chunking

In [ ]:
# Chunking-Parameter
CHUNK_SIZE = 1500
OVERLAP = 200

Chunk-Größe: 1500 Zeichen
Overlap: 200 Zeichen


## Markdown-Datei einlesen

In [10]:
# Markdown-Datei einlesen
with open(markdown_path, "r", encoding="utf-8") as f:
    markdown_text = f.read()

print(f"Datei geladen: {markdown_path}")
print(f"Gesamtlänge: {len(markdown_text)} Zeichen")
print(f"Gesamtlänge: {len(markdown_text.split())} Wörter")

Datei geladen: docs/extracted_content_geschichtsbuch.md
Gesamtlänge: 466497 Zeichen
Gesamtlänge: 73166 Wörter


## Character Splitting implementieren

In [ ]:
def character_split(text, chunk_size, overlap):
    chunks = []
    start = 0
    text_length = len(text)
    
    while start < text_length:
        
        end = start + chunk_size # Ende des Chunks
        
        chunk = text[start:end]
        chunks.append(chunk) # Chunk extrahieren
        
        start = end - overlap # Nächste Startposition: Ende minus Overlap
        
        # Verhindere Endlosschleife bei sehr kleinen Texten
        if start + chunk_size >= text_length and start < text_length:
            # Letzter Chunk
            chunks.append(text[start:])
            break
    
    return chunks

# Text in Chunks aufteilen
chunks = character_split(markdown_text, CHUNK_SIZE, OVERLAP)

print(f"\n{'='*60}")
print(f"Text wurde in {len(chunks)} Chunks aufgeteilt")
print(f"{'='*60}")
print(f"\nErste 3 Chunks (Vorschau):")
print(f"-"*60)

for i, chunk in enumerate(chunks[:3]):
    print(f"\n--- Chunk {i+1} (Länge: {len(chunk)} Zeichen) ---")
    print(chunk[:200] + "..." if len(chunk) > 200 else chunk)


Text wurde in 359 Chunks aufgeteilt

Erste 3 Chunks (Vorschau):
------------------------------------------------------------

--- Chunk 1 (Länge: 1500 Zeichen) ---
## www.e-rara.ch

Die Plattform e-rara.ch macht die in Schweizer Bibliotheken vorhandenen Drucke online verfügbar. Das Spektrum reicht von Büchern über Karten bis zu illustrierten Materialien - von de...

--- Chunk 2 (Länge: 1500 Zeichen) ---
n the title information for each document individually. For further information please refer to the terms of use on [Link]

Conditions d'utilisation Ce document numérique peut être téléchargé gratuite...

--- Chunk 3 (Länge: 1500 Zeichen) ---
Sutrrs 'Schweizer Gelchichte " bearbeitet van

## I. LrvLler,

Lrhrrr an ürr untern Kealschulr in Lufrrn.

Mit I &gt; 5 Textbildern , farbiger Wappentafel und 8 farbigen Geschichtskarten der Schweiz n...


## Statistiken der Chunks

In [12]:
# Statistiken über die Chunks
chunk_lengths = [len(chunk) for chunk in chunks]

print(f"Anzahl Chunks: {len(chunks)}")
print(f"Durchschnittliche Chunk-Länge: {sum(chunk_lengths) / len(chunk_lengths):.0f} Zeichen")
print(f"Kürzester Chunk: {min(chunk_lengths)} Zeichen")
print(f"Längster Chunk: {max(chunk_lengths)} Zeichen")

# Überprüfe mit Tokenizer (falls geladen)
try:
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
    
    # Teste ersten Chunk mit Prefix
    sample_chunk = f"passage: {chunks[0]}"
    tokens = tokenizer(sample_chunk, return_tensors='pt', truncation=False)
    num_tokens = tokens['input_ids'].shape[1]
    
    print(f"\n{'='*60}")
    print(f"Token-Check für ersten Chunk:")
    print(f"{'='*60}")
    print(f"Zeichen: {len(chunks[0])}")
    print(f"Tokens: {num_tokens}")
    print(f"Status: {'✅ OK (unter 512)' if num_tokens <= 512 else '❌ ZU LANG'}")
except Exception as e:
    print(f"\nTokenizer nicht verfügbar: {e}")

Anzahl Chunks: 359
Durchschnittliche Chunk-Länge: 1499 Zeichen
Kürzester Chunk: 1097 Zeichen
Längster Chunk: 1500 Zeichen

Token-Check für ersten Chunk:
Zeichen: 1500
Tokens: 339
Status: ✅ OK (unter 512)


## Chunks speichern

In [16]:
import json
from pathlib import Path

# Dateiname basierend auf dem Quell-Dokument
output_file = json_chunk_path

# Chunks mit Metadaten speichern
chunks_data = {
    "source_file": markdown_path,
    "chunk_size": CHUNK_SIZE,
    "overlap": OVERLAP,
    "total_chunks": len(chunks),
    "chunks": [
        {
            "chunk_id": i,
            "text": chunk,
            "length": len(chunk)
        }
        for i, chunk in enumerate(chunks)
    ]
}

# Als JSON speichern
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(chunks_data, f, ensure_ascii=False, indent=2)

print(f"✅ {len(chunks)} Chunks gespeichert in: {output_file}")
print(f"Dateigröße: {Path(output_file).stat().st_size / 1024:.1f} KB")

✅ 359 Chunks gespeichert in: docs/chunks_geschichtsbuch.json
Dateigröße: 565.0 KB


## Chunks wieder einlesen (zum Testen)

In [17]:
# Chunks wieder einlesen
with open(output_file, "r", encoding="utf-8") as f:
    loaded_data = json.load(f)

# Metadaten anzeigen
print("Geladene Daten:")
print(f"  Quelle: {loaded_data['source_file']}")
print(f"  Chunk-Größe: {loaded_data['chunk_size']}")
print(f"  Overlap: {loaded_data['overlap']}")
print(f"  Anzahl Chunks: {loaded_data['total_chunks']}")

# Die eigentlichen Text-Chunks extrahieren
loaded_chunks = [chunk_data["text"] for chunk_data in loaded_data["chunks"]]

print(f"\n✅ {len(loaded_chunks)} Chunks erfolgreich geladen")
print(f"\nBeispiel - Erster Chunk (erste 150 Zeichen):")
print(loaded_chunks[0][:150] + "...")

Geladene Daten:
  Quelle: docs/extracted_content_geschichtsbuch.md
  Chunk-Größe: 1500
  Overlap: 200
  Anzahl Chunks: 359

✅ 359 Chunks erfolgreich geladen

Beispiel - Erster Chunk (erste 150 Zeichen):
## www.e-rara.ch

Die Plattform e-rara.ch macht die in Schweizer Bibliotheken vorhandenen Drucke online verfügbar. Das Spektrum reicht von Büchern übe...
